In [1]:
from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.aa import AA_TEST
from hypex.reporters import AADictReporter

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(),
        "treat": TreatmentRole(),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole()
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.roles

{'user_id': Info,
 'treat': Treatment,
 'pre_spends': Target,
 'post_spends': Target,
 'signup_month': Feature,
 'age': Feature,
 'gender': Feature,
 'industry': Feature}

In [4]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [5]:
AADictReporter().report(result)

{'random_state': None,
 'B pre_spends control mean': 487.0404,
 'B pre_spends test mean': 487.1471,
 'B pre_spends difference': 0.1067000000000462,
 'B pre_spends difference %': 0.021907833518541864,
 'B post_spends control mean': 451.7942666666667,
 'B post_spends test mean': 452.5348444444444,
 'B post_spends difference': 0.7405777777776734,
 'B post_spends difference %': 0.16391925095500248,
 'B control size': 5000.0,
 'B test size': 5000.0,
 'B control size %': 50.0,
 'B test size %': 50.0,
 'TTest p-value': p-value    0.562399
 dtype: float64,
 'TTest pass': pass    0.0
 dtype: float64,
 'KSTest p-value': p-value    0.749007
 dtype: float64,
 'KSTest pass': pass    0.0
 dtype: float64,
 'mean test score': p-value    0.686804
 dtype: float64}

In [6]:
result.analysis_tables

{"GroupSizes┴┴['AASplitter||']":                      B
 control size    5000.0
 test size       5000.0
 control size %    50.0
 test size %       50.0,
 "GroupDifference┴┴['AASplitter||'] ┆pre_spends┆":                                   B
 pre_spends control mean  487.040400
 pre_spends test mean     487.147100
 pre_spends difference      0.106700
 pre_spends difference %    0.021908,
 "TTest┴┴['AASplitter||'] ┆pre_spends┆":   group  statistic  p-value   pass
 0     B  -0.282703  0.77741  False,
 "KSTest┴┴['AASplitter||'] ┆pre_spends┆":   group  statistic   p-value   pass
 0     B     0.0098  0.970009  False,
 "GroupDifference┴┴['AASplitter||'] ┆post_spends┆":                                    B
 post_spends control mean  451.794267
 post_spends test mean     452.534844
 post_spends difference      0.740578
 post_spends difference %    0.163919,
 "TTest┴┴['AASplitter||'] ┆post_spends┆":   group  statistic   p-value   pass
 0     B  -0.939713  0.347387  False,
 "KSTest┴┴['AASplitter||

In [7]:
from hypex.experiments.ab import AB_TEST

test = AB_TEST
ed = ExperimentData(data)
result = test.execute(ed)

IndexError: list index out of range

In [ ]:
result.analysis_tables['ABAnalyzer╰╰']

In [ ]:
from hypex.experiments.homogeneity import HOMOGENEITY_TEST

test = HOMOGENEITY_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [ ]:
result.analysis_tables

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from hypex.comparators import TTest, KSTest, GroupSizes, GroupDifference
from hypex.utils import SpaceEnum
from hypex.splitters import AASplitter
from hypex.analyzers import OneAASplitAnalyzer
from hypex.experiments import CycledExperiment, Experiment, OnRoleExperiment

aa = Experiment(
    executors=[
        AASplitter(),
        OnRoleExperiment(
            executors=[
                GroupSizes(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                GroupDifference(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                TTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                KSTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
            ],
            role=TargetRole(),
        )
    ]
)
experiment = CycledExperiment(n_iterations=20, inner_executor=aa, analyzer=OneAASplitAnalyzer())
ed = ExperimentData(data)
res = experiment.execute(ed)

In [ ]:
res.additional_fields

In [ ]:
res.analysis_tables

In [ ]:
res_analyzer = OneAASplitAnalyzer().execute(res)

In [ ]:
res_analyzer.analysis_tables['OneAASplitAnalyzer╰╰']

In [ ]:
# analyser version

from hypex.dataset import StatisticRole

new_res = {}
fields = ['pre_spends', 'post_spends'] # получаем столбцы по TargetRole
tests = ['KSTest', 'TTest']
for j in fields:
    for i in list(res.analysis_tables.keys()):
        if j in i and i.split('╰╰')[0] in tests:
            test = i.split('╰╰')[0]
            if j not in new_res:
                new_res[j] = {test: []}
            elif test not in new_res[j]:
                new_res[j].update({test: []})
            new_res[j][test].append(list(res.analysis_tables[i].data['pass'])[0])
    new_res[j] = Dataset.from_dict(new_res[j], roles={}).mean()
    new_res[j].add_column(new_res[j].apply(
                lambda x: int(0.8 * 0.05 <= x['TTest'] <= 1.2 * 0.05), {'TTest passed': StatisticRole()}, axis=1
            ))
    new_res[j].add_column(new_res[j].apply(
                lambda x: int(0.8 * 0.05 <= x['KSTest'] <= 1.2 * 0.05), {'KSTest passed': StatisticRole()}, axis=1
            ))
new_res

In [ ]:
import pandas as pd

t = pd.read_csv("data.csv")
t.sample(frac=1, random_state=None)

In [ ]:
from hypex.hypotheses.hypothesis import Hypothesis
ed, exp  = Hypothesis("test_config.json").execute()

In [ ]:
res = exp.execute(ed)

In [ ]:
exp.executors[1].executors[0].space

In [ ]:
res.analysis_tables

In [ ]:
config = {
  "dataset": {
    "path": "data.csv",
    "roles": {
      "role_names": [
        "Info",
        "Treatment",
        "Target",
        "Target",
      ],
      "columns": [
        "user_id",
        "treat",
        "pre_spends",
        "post_spends"
      ]
    },
    "backend": "pandas"
  },
  "experiment": {
    "AASplitter": {
      "control_size": 0.5,
      "random_state": None
    },
    "OnRoleExperiment": {
      "executors": {
        "GroupDifference": {
          "grouping_role": "Treatment",
          "space": "additional"
        }
      },
      "role": "Target"
    }
  },
  "report": {}
} 

ed, exp  = Hypothesis(config).execute()

In [ ]:
res = exp.execute(ed)

In [ ]:
res.analysis_tables